In [6]:
import sys
sys.path.append("src/")
from cgra import *
from sat_to_csv import *
from kernels import *

In [7]:
benchmark_name = "gemm_majo"  # supported benchmarks: conv3d, bicg
version = "v3"
cgra_grid = 4

In [8]:
import numpy as np

def gemm_majo(rows_a, cols_a, cols_b, matrix_a, matrix_b):
    """
    General Matrix Multiplication implementation in Python.
    
    Args:
        rows_a: Number of rows in matrix A
        cols_a: Number of columns in matrix A
        cols_b: Number of columns in matrix B
        matrix_a: Input matrix A as a flat list or 1D array
        matrix_b: Input matrix B as a flat list or 1D array
        
    Returns:
        A new output matrix as a 1D array representing the result of A×B
    """
    # Initialize output matrix with zeros
    output = [0] * (rows_a * cols_b)
    
    for i in range(rows_a):
        for j in range(cols_b):
            sum_val = 0
            for k in range(cols_a):
                # Convert 2D indices to 1D array index
                a_idx = i * cols_a + k
                b_idx = k * cols_b + j
                sum_val += matrix_a[a_idx] * matrix_b[b_idx]
            output[i * cols_b + j] = sum_val
            
    return output

rows_a = 64
cols_a = 64
cols_b = 4
matrix_a = np.random.randint(0, 10, (rows_a, cols_a)).flatten()
matrix_b = np.random.randint(0, 10, (cols_a, cols_b)).flatten()
python_output = gemm_majo(rows_a, cols_a, cols_b, matrix_a, matrix_b)

In [ ]:
# Print the results
if benchmark_name == "gemm_majo":
    print("A:")
    print(matrix_a)
    print("B:")
    print(matrix_b)
    print("\nOutput Tensor:")
    print(python_output)

In [ ]:
if benchmark_name == "gemm_majo":
    # Kernels will be located in a folder with the kernel's name
    kernel_name = "examples_compigra1/"+benchmark_name

    # Create a new folder (if it does not exist already) and add the minimum necessary files to run it
    kernel_new(kernel_name)

    # If you have a assembly in the SAT-MapIt format, you can convert it to a csv
    convert( kernel_name+"/out_" + str(cgra_grid) + "_" + version + ".sat", kernel_name+"/instructions.csv" )

    # To test this kernel we will fill the memory with two random arrays
    kernel_clear_memory(kernel_name)

    # initialize memory cmd·
    # concatenate matrix_a to be 64 x 64
    A = np.zeros((64, 64)).astype(int)
    A = A.flatten()
    A[:rows_a*cols_a] = matrix_a
    print(A)

    # concatenate matrix_b to be 64 x 64
    B = np.zeros((64, 64)).astype(int)
    B = B.flatten()
    B[:cols_a*cols_b] = matrix_b
    print(B)
    kernel_add_memory_region(kernel_name, 51328, np.concatenate((np.array([rows_a, cols_a, cols_b]), A, B.flatten())))


    import cgra
    # Simulate the execution of the kernel in CGRA
    cgra.run( kernel_name, pr=["ROUT", "R0", "R1", "INST"], limit=200000)